In [2]:
import pandas as pd

feat_path = "C:/Users/kiran/Cheminformatics_course/ADMET_Prediction/DATA/FEATURES/hia_features.csv"
tgt_path  = "C:/Users/kiran/Cheminformatics_course/ADMET_Prediction/DATA/FEATURES/hia_targets.csv"

X = pd.read_csv(feat_path)
y = pd.read_csv(tgt_path)

print(X.shape)
print(y.shape)
print(X.columns[:5])

(3552, 2058)
(3552, 3)
Index(['canonical_smiles', 'MolWt', 'MolLogP', 'TPSA', 'NumHDonors'], dtype='object')


In [3]:
import pandas as pd

feat_path = "C:/Users/kiran/Cheminformatics_course/ADMET_Prediction/DATA/FEATURES/hia_features.csv"
tgt_path  = "C:/Users/kiran/Cheminformatics_course/ADMET_Prediction/DATA/FEATURES/hia_targets.csv"

X = pd.read_csv(feat_path)
y = pd.read_csv(tgt_path)

print("X:", X.shape)
print("y:", y.shape)
print("Features:", X.columns[:10])
print("Targets:", y.columns)

X: (3552, 2058)
y: (3552, 3)
Features: Index(['canonical_smiles', 'MolWt', 'MolLogP', 'TPSA', 'NumHDonors',
       'NumHAcceptors', 'NumRotatableBonds', 'FractionCSP3', 'HeavyAtomCount',
       'RingCount'],
      dtype='object')
Targets: Index(['canonical_smiles', 'logPapp_median', 'Papp_median_cm_s'], dtype='object')


In [4]:
import pandas as pd

feat_path = "C:/Users/kiran/Cheminformatics_course/ADMET_Prediction/DATA/FEATURES/hia_features.csv"
tgt_path  = "C:/Users/kiran/Cheminformatics_course/ADMET_Prediction/DATA/FEATURES/hia_targets.csv"

# Load
X_raw = pd.read_csv(feat_path)
y_raw = pd.read_csv(tgt_path)

# Drop SMILES columns
X = X_raw.drop(columns=["canonical_smiles"])
y = y_raw["logPapp_median"]

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (3552, 2057)
y shape: (3552,)


In [5]:
from sklearn.model_selection import train_test_split

# First split: train vs temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42
)

# Second split: validation vs test (15% each)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42
)

print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)

Train: (2486, 2057) (2486,)
Val:   (533, 2057) (533,)
Test:  (533, 2057) (533,)


In [6]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, r2_score

# LightGBM model (optimized configuration)
model_lgb_hia = lgb.LGBMRegressor(
    n_estimators=1500,
    learning_rate=0.03,
    num_leaves=120,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
)

print("Training LightGBM Absorption model...")

model_lgb_hia.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric="rmse",
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)

# Predict on test set
pred = model_lgb_hia.predict(X_test)

# Evaluate
mae = mean_absolute_error(y_test, pred)
r2 = r2_score(y_test, pred)

print("\n⭐️ Absorption LightGBM Model Results:")
print("MAE:", mae)
print("R² :", r2)

Training LightGBM Absorption model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028882 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3702
[LightGBM] [Info] Number of data points in the train set: 2486, number of used features: 1285
[LightGBM] [Info] Start training from score -3.805719
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

In [7]:
model_lgb_hia2 = lgb.LGBMRegressor(
    n_estimators=3000,
    learning_rate=0.015,
    num_leaves=256,
    max_depth=-1,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_alpha=1.0,
    reg_lambda=1.0,
    min_child_samples=10,
    random_state=42
)

print("Training Improved LGBM Absorption model...")

model_lgb_hia2.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric="rmse",
    callbacks=[lgb.early_stopping(stopping_rounds=80)]
)

pred2 = model_lgb_hia2.predict(X_test)

from sklearn.metrics import mean_absolute_error, r2_score
mae2 = mean_absolute_error(y_test, pred2)
r2_2 = r2_score(y_test, pred2)

print("\n⭐️ Improved Absorption Model Results:")
print("MAE:", mae2)
print("R² :", r2_2)

Training Improved LGBM Absorption model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029849 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4628
[LightGBM] [Info] Number of data points in the train set: 2486, number of used features: 1748
[LightGBM] [Info] Start training from score -3.805719
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 80 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

In [8]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error, r2_score

et = ExtraTreesRegressor(
    n_estimators=1500,
    max_features='sqrt',
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

print("Training ExtraTrees Absorption model...")
et.fit(X_train, y_train)

pred_et = et.predict(X_test)

mae_et = mean_absolute_error(y_test, pred_et)
r2_et = r2_score(y_test, pred_et)

print("\n⭐️ ExtraTrees Absorption Results:")
print("MAE:", mae_et)
print("R² :", r2_et)

Training ExtraTrees Absorption model...

⭐️ ExtraTrees Absorption Results:
MAE: 1.0805374984855909
R² : 0.6904987331393772


In [9]:
et2 = ExtraTreesRegressor(
    n_estimators=2000,
    max_features=0.3,   # more focused splits
    min_samples_split=2,
    min_samples_leaf=1,
    bootstrap=False,
    random_state=42,
    n_jobs=-1
)

print("Training ExtraTrees Tuned model...")
et2.fit(X_train, y_train)

pred2 = et2.predict(X_test)

from sklearn.metrics import mean_absolute_error, r2_score
print("\n⭐️ Tuned ExtraTrees Absorption Results:")
print("MAE:", mean_absolute_error(y_test, pred2))
print("R² :", r2_score(y_test, pred2))

Training ExtraTrees Tuned model...

⭐️ Tuned ExtraTrees Absorption Results:
MAE: 1.0676106738312876
R² : 0.6550534378324944


In [12]:
import pandas as pd

path = r"C:/Users/kiran/Cheminformatics_course/ADMET_Prediction/DATA/chembl_hia_filtered_expanded.csv"

df = pd.read_csv(path)
print(df.shape)
print(df.columns)

(8496, 7)
Index(['assay_chembl_id', 'molecule_chembl_id', 'standard_value',
       'standard_units', 'standard_type', 'canonical_smiles',
       'is_absorb_type'],
      dtype='object')


In [13]:
df['standard_type'].unique()
df['standard_units'].unique()
df['standard_value'].describe()

count    7.875000e+03
mean     4.814393e+05
std      5.259371e+06
min     -7.800000e+00
25%      1.100000e+00
50%      8.500000e+00
75%      3.654500e+01
max      1.190000e+08
Name: standard_value, dtype: float64

In [14]:
df2 = df[(df['standard_value'] > 0) & (df['standard_value'] < 150)]

In [15]:
df2['standard_value'].describe()

count    6.793000e+03
mean     2.166837e+01
std      3.053581e+01
min      4.000000e-08
25%      1.450000e+00
50%      8.000000e+00
75%      2.700000e+01
max      1.480000e+02
Name: standard_value, dtype: float64

In [16]:
df2.shape

(6793, 7)

In [17]:
df['standard_type'].unique()

array(['PCaco-2', 'Papp', 'Papp e-6', 'Permeability',
       'Permeability coefficient', 'Absorption', 'Pcaco2',
       'Cell permeation', 'Papps', 'Permeability rate', 'permeability',
       'logPapp', 'LogPapp', 'Papp A to B (mean)',
       'Recovery A to B (mean)', 'Papp B to A (mean)',
       'Recovery B to A (mean)', 'Radioactive absorption',
       'Permeability limited', 'Caco-2 A-B', 'Caco-2 Papp',
       'Caco-2 permeability', 'Drugabsorption'], dtype=object)

In [18]:
absorption_terms = [
    "Absorption", "Drugabsorption", "Radioactive absorption"
]

df_abs = df[df["standard_type"].isin(absorption_terms)].copy()

print(df_abs.shape)
print(df_abs["standard_value"].describe())

(209, 7)
count    189.000000
mean      63.625545
std       90.856948
min        0.009000
25%       17.000000
50%       50.000000
75%       93.000000
max      725.000000
Name: standard_value, dtype: float64


In [2]:
import pandas as pd

# Path to your dataset
file_path = r"C:/Users/kiran/Cheminformatics_course/ADMET_Prediction/DATA/chembl_hia_final_aggregated.csv"

# Load Excel file
hia_df = pd.read_csv(file_path)

# Basic info
print("✅ Dataset loaded successfully!")
print("Shape:", hia_df.shape)
print("\nColumns:", hia_df.columns.tolist())

# Quick look at the first few rows
print("\nPreview:")
display(hia_df.head())

# Summary of the 'standard_value' column (important target)
if 'standard_value' in hia_df.columns:
    print("\nTarget column statistics:")
    print(hia_df['standard_value'].describe())

✅ Dataset loaded successfully!
Shape: (3552, 8)

Columns: ['canonical_smiles', 'logPapp_median', 'Papp_median_cm_s', 'molecule_chembl_id', 'standard_type', 'standard_units', 'assay_chembl_id', 'conv_reason']

Preview:


,canonical_smiles,logPapp_median,Papp_median_cm_s,molecule_chembl_id,standard_type,standard_units,assay_chembl_id,conv_reason
0,C#CC(C)(C)N1CCC(CC(=O)NO)(NC(=O)c2ccc(OCc3cc(C...,0.505150,3.200000,CHEMBL207709,Papp,ucm/s,CHEMBL853098,cm_per_s
1,C#CCN(CC#C)C(=O)[C@@]1(c2ccc3c(c2)OCCO3)C[C@H]1CN,-5.283997,0.000005,CHEMBL402441,Papp,nm/s,CHEMBL930503,nm_per_s
2,C#CCN(CC#C)C(=O)[C@@]1(c2cccs2)C[C@H]1CN,-4.602060,0.000025,CHEMBL513248,Papp,nm/s,CHEMBL948483,nm_per_s
3,C#CCN(CC#C)C(=O)[C@@]1(c2ccsc2)C[C@H]1CN,-4.785156,0.000016,CHEMBL257388,Papp,nm/s,CHEMBL931359,nm_per_s
4,C#CCN(CC)C(=O)[C@@]1(c2cccs2)C[C@H]1CN,-4.686133,0.000021,CHEMBL472253,Papp,nm/s,CHEMBL948483,nm_per_s


In [4]:
import pandas as pd

# 1) Already-loaded targets (you just did this)
targets = hia_df[['canonical_smiles', 'logPapp_median']].copy()

# 2) Load features file
feat_path = r"C:/Users/kiran/Cheminformatics_course/ADMET_Prediction/DATA/FEATURES/hia_features.csv"
features = pd.read_csv(feat_path)

print("Features shape:", features.shape)
print("Feature columns (first 10):", features.columns[:10].tolist())

# 3) Merge on SMILES
merged = pd.merge(features, targets, on="canonical_smiles", how="inner")

print("\nMerged shape:", merged.shape)
print("Columns:", merged.columns[:15].tolist())

Features shape: (3552, 2058)
Feature columns (first 10): ['canonical_smiles', 'MolWt', 'MolLogP', 'TPSA', 'NumHDonors', 'NumHAcceptors', 'NumRotatableBonds', 'FractionCSP3', 'HeavyAtomCount', 'RingCount']

Merged shape: (3552, 2059)
Columns: ['canonical_smiles', 'MolWt', 'MolLogP', 'TPSA', 'NumHDonors', 'NumHAcceptors', 'NumRotatableBonds', 'FractionCSP3', 'HeavyAtomCount', 'RingCount', 'FP_0', 'FP_1', 'FP_2', 'FP_3', 'FP_4']


In [5]:
# Remove smiles column
X = merged.drop(columns=["canonical_smiles", "logPapp_median"])
y = merged["logPapp_median"]

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (3552, 2057)
y shape: (3552,)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42
)

print("Train:", X_train.shape, y_train.shape)
print("Test :", X_test.shape, y_test.shape)

Train: (3019, 2057) (3019,)
Test : (533, 2057) (533,)


In [8]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, r2_score

model = LGBMRegressor(
    n_estimators=1200,
    learning_rate=0.01,
    num_leaves=120,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train, y_train)

# predict
pred = model.predict(X_test)

print("MAE:", mean_absolute_error(y_test, pred))
print("R² :", r2_score(y_test, pred))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039034 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4033
[LightGBM] [Info] Number of data points in the train set: 3019, number of used features: 1449
[LightGBM] [Info] Start training from score -3.780999
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [9]:
model = LGBMRegressor(
    n_estimators=2000,
    learning_rate=0.005,
    num_leaves=150,
    max_depth=-1,
    subsample=0.9,
    colsample_bytree=0.9,
    min_child_samples=20,
    random_state=42
)

model.fit(X_train, y_train)

pred = model.predict(X_test)

print("MAE:", mean_absolute_error(y_test, pred))
print("R² :", r2_score(y_test, pred))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027561 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4033
[LightGBM] [Info] Number of data points in the train set: 3019, number of used features: 1449
[LightGBM] [Info] Start training from score -3.780999
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [1]:
from pathlib import Path
import joblib
import numpy as np
import pandas as pd

# PATHS
(base := Path("C:/Users/kiran/cheminformatics_course/HIA_model_package"))
model_path = base / "models/hia_lgbm_model.pkl"
cols_path = base / "models/feature_columns_numeric.pkl"
fp_bool_path = base / "models/hia_train_fp_bool.npy"
knn_path = base / "models/hia_knn_ad_jaccard.pkl"

# LOAD EVERYTHING
model = joblib.load(model_path)
feature_cols = joblib.load(cols_path)
train_fp_matrix = np.load(fp_bool_path)
knn_ad = joblib.load(knn_path)

print("Artifacts loaded:")
print("Model ✔️")
print("Feature columns ✔️ (", len(feature_cols), ")")
print("FP matrix ✔️", train_fp_matrix.shape)
print("AD model ✔️")

Artifacts loaded:
Model ✔️
Feature columns ✔️ ( 2057 )
FP matrix ✔️ (2841, 2048)
AD model ✔️


In [2]:
predict_hia("CC(=O)NC1=CC=C(C=C1)O")
explain_hia("CC(=O)NC1=CC=C(C=C1)O")

NameError: name 'predict_hia' is not defined

In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, rdMolDescriptors
from rdkit.DataStructs import ConvertToNumpyArray
import numpy as np
import pandas as pd

def featurize_smiles(smiles_list):
    """
    Takes a SMILES (str) or list of SMILES and returns:
    - X_num: DataFrame with numeric features, columns aligned to feature_cols
    - fps_bool: numpy bool array with Morgan fingerprints (for AD check)
    """
    if isinstance(smiles_list, str):
        smiles_list = [smiles_list]

    rows = []
    fps = []
    n_bits = train_fp_matrix.shape[1]  # match training FP size

    # pre-create descriptor list
    desc_list = Descriptors.descList
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            raise ValueError(f"Invalid SMILES: {smi}")

        desc_vals = {}
        for name, func in desc_list:
            try:
                desc_vals[name] = func(mol)
            except Exception:
                desc_vals[name] = np.nan

        rows.append(desc_vals)

        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=n_bits)
        arr = np.zeros((n_bits,), dtype=bool)
        ConvertToNumpyArray(fp, arr)
        fps.append(arr)

    df_all = pd.DataFrame(rows)

    # ensure all feature columns exist
    for col in feature_cols:
        if col not in df_all.columns:
            df_all[col] = np.nan  # placeholder if your training used extra features

    X_num = df_all[feature_cols]
    fps_bool = np.vstack(fps)

    return X_num, fps_bool

In [4]:
def predict_hia(smiles):
    """
    Returns prediction + probability for a SMILES.
    """
    X_num, fps_bool = featurize_smiles(smiles)
    prob = float(model.predict_proba(X_num)[:, 1][0])  # class 1 prob
    pred_label = int(model.predict(X_num)[0])

    return {
        "smiles": smiles,
        "prob_high_absorption": prob,
        "pred_class": pred_label  # 1 = high, 0 = low (assuming this mapping)
    }

In [5]:
result = predict_hia("CC(=O)NC1=CC=C(C=C1)O")
result

[13:15:45] DEPRECATION WARNING: please use MorganGenerator
C:\Users\kiran\AppData\Local\Temp\ipykernel_15016\550108114.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all[col] = np.nan  # placeholder if your training used extra features
C:\Users\kiran\AppData\Local\Temp\ipykernel_15016\550108114.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all[col] = np.nan  # placeholder if your training used extra features
C:\Users\kiran\AppData\Local\Temp\ipykernel_15016\550108114.py:46: PerformanceWarning: DataFrame is hig

AttributeError: 'LGBMRegressor' object has no attribute 'predict_proba'

In [6]:
def predict_hia(smiles, threshold=0.5):
    X_num, fps_bool = featurize_smiles(smiles)
    
    score = float(model.predict(X_num)[0])  # regression output
    
    pred_label = 1 if score >= threshold else 0

    return {
        "smiles": smiles,
        "absorption_score": score,
        "pred_class": pred_label,   # 1 = High, 0 = Low
        "threshold_used": threshold
    }

In [7]:
def explain_hia(smiles, threshold=0.5, distance_threshold=0.3):
    X_num, fps_bool = featurize_smiles(smiles)

    score = float(model.predict(X_num)[0])
    label_text = "HIGH absorption" if score >= threshold else "LOW absorption"

    # AD check
    dists, idx = knn_ad.kneighbors(fps_bool, n_neighbors=1)
    d = float(dists[0, 0])
    inside = d <= distance_threshold

    print(f"SMILES: {smiles}")
    print(f"Absorption score: {score:.3f}")
    print(f"Prediction: {label_text} (threshold = {threshold})")
    print(f"AD distance: {d:.3f}")
    print(f"Inside AD: {inside}")

    # Show top descriptors the model used (optional)
    top_feats = X_num.iloc[0].sort_values(ascending=False).head(5)
    print("\nTop 5 descriptor values:")
    for name, val in top_feats.items():
        print(f"  {name}: {val:.3f}")

In [14]:
result = predict_hia("CC(C)CC1=CC=C(C=C1)C(C)C(=O)O")
result

[14:18:25] DEPRECATION WARNING: please use MorganGenerator
C:\Users\kiran\AppData\Local\Temp\ipykernel_15016\550108114.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all[col] = np.nan  # placeholder if your training used extra features
C:\Users\kiran\AppData\Local\Temp\ipykernel_15016\550108114.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all[col] = np.nan  # placeholder if your training used extra features
C:\Users\kiran\AppData\Local\Temp\ipykernel_15016\550108114.py:46: PerformanceWarning: DataFrame is hig

{'smiles': 'CC(C)CC1=CC=C(C=C1)C(C)C(=O)O',
 'absorption_score': -5.641138738464394,
 'pred_class': 0,
 'threshold_used': 0.5}

In [15]:
explain_hia("CC(C)CC1=CC=C(C=C1)C(C)C(=O)O")

[14:18:38] DEPRECATION WARNING: please use MorganGenerator
C:\Users\kiran\AppData\Local\Temp\ipykernel_15016\550108114.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all[col] = np.nan  # placeholder if your training used extra features
C:\Users\kiran\AppData\Local\Temp\ipykernel_15016\550108114.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all[col] = np.nan  # placeholder if your training used extra features
C:\Users\kiran\AppData\Local\Temp\ipykernel_15016\550108114.py:46: PerformanceWarning: DataFrame is hig

SMILES: CC(C)CC1=CC=C(C=C1)C(C)C(=O)O
Absorption score: -5.641
Prediction: LOW absorption (threshold = 0.5)
AD distance: 0.000
Inside AD: True

Top 5 descriptor values:
  MolWt: 206.285
  TPSA: 37.300
  HeavyAtomCount: 15.000
  NumRotatableBonds: 4.000
  MolLogP: 3.073
